In [1]:
import pandas as pd
import numpy as np 
ESG=pd.read_csv('../data/ESG_Score.csv')
hist_scope=pd.read_csv('../data/raw/hist_scope.csv')
hist_score=pd.read_csv('../data/raw/hist_score.csv')

In [2]:
hist_score

,Isin,Year,SR001,SR002,SR003,SR004
0,US0079031078,2018,6.070000172,6.841240406,4.237225056,7.072086811
1,US0326541051,2018,4.53000021,4.951658726,2.288488626,6.702093601
2,US0382221051,2018,6.900000095,6.190028667,6.683382988,8.140346527
3,US11135F1012,2018,2.029999971,0,1.427484035,7.473958492
4,US29355A1079,2018,3.069999933,0.130537882,6.115842342,5.971273899
...,...,...,...,...,...,...
1735,US0395872098,2022,1.519999981,0,0.806498468,6.890572548
1736,US54405Q2093,2022,1.25999999,0,0.582704067,5.70973444
1737,US3073597037,2022,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
1738,CA35954F1080,2022,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable


In [3]:
hist_score = hist_score.replace(['#N/A', '#N/A Field Not Applicable'], np.nan)
numeric_columns = ['SR001', 'SR002', 'SR003', 'SR004']
hist_score[numeric_columns] = hist_score[numeric_columns].astype(float)
hist_score

,Isin,Year,SR001,SR002,SR003,SR004
0,US0079031078,2018,6.07,6.841240,4.237225,7.072087
1,US0326541051,2018,4.53,4.951659,2.288489,6.702094
2,US0382221051,2018,6.90,6.190029,6.683383,8.140347
3,US11135F1012,2018,2.03,0.000000,1.427484,7.473958
4,US29355A1079,2018,3.07,0.130538,6.115842,5.971274
...,...,...,...,...,...,...
1735,US0395872098,2022,1.52,0.000000,0.806498,6.890573
1736,US54405Q2093,2022,1.26,0.000000,0.582704,5.709734
1737,US3073597037,2022,NaN,NaN,NaN,NaN
1738,CA35954F1080,2022,NaN,NaN,NaN,NaN


In [4]:
new_rows = []
for index, row in ESG.iterrows():
    isin = row['ISIN']
    esg_score = row['ESG SCORE']
    env_score = row['ENVIRONMENTAL SCORE']
    social_score = row['SOCIAL SCORE']
    governance_score = row['GOVERNANCE SCORE']
    
    new_row = {
        'Isin': isin,
        'Year': 2023,
        'SR001': esg_score,
        'SR002': env_score,
        'SR003': social_score,
        'SR004': governance_score
    }
    new_rows.append(new_row)

new_rows_df = pd.DataFrame(new_rows)
hist_score = pd.concat([hist_score, new_rows_df], ignore_index=True)


In [5]:
hist_score.head()

,Isin,Year,SR001,SR002,SR003,SR004
0,US0079031078,2018,6.07,6.841240,4.237225,7.072087
1,US0326541051,2018,4.53,4.951659,2.288489,6.702094
2,US0382221051,2018,6.90,6.190029,6.683383,8.140347
3,US11135F1012,2018,2.03,0.000000,1.427484,7.473958
4,US29355A1079,2018,3.07,0.130538,6.115842,5.971274


In [6]:

hist_2021 = hist_score[hist_score['Year'] == 2021]

combined = pd.merge(hist_2021[['Isin', 'SR001', 'SR002', 'SR003', 'SR004']], 
                    ESG[['ISIN', 'ESG SCORE', 'ENVIRONMENTAL SCORE', 'SOCIAL SCORE', 'GOVERNANCE SCORE']], 
                    left_on='Isin', right_on='ISIN', how='left')

combined['Trend_ESG_SR001'] = combined['ESG SCORE'] - combined['SR001']
combined['Trend_ENV_SR002'] = combined['ENVIRONMENTAL SCORE'] - combined['SR002']
combined['Trend_SOC_SR003'] = combined['SOCIAL SCORE'] - combined['SR003']
combined['Trend_GOV_SR004'] = combined['GOVERNANCE SCORE'] - combined['SR004']

historical_score = pd.merge(hist_score, combined[['Isin', 'Trend_ESG_SR001', 'Trend_ENV_SR002', 'Trend_SOC_SR003', 'Trend_GOV_SR004']], on='Isin', how='left')

historical_score

,Isin,Year,SR001,SR002,SR003,SR004,Trend_ESG_SR001,Trend_ENV_SR002,Trend_SOC_SR003,Trend_GOV_SR004
0,US0079031078,2018,6.070000,6.841240,4.237225,7.072087,0.190441,0.230901,0.417430,-0.192367
1,US0326541051,2018,4.530000,4.951659,2.288489,6.702094,0.866660,1.043274,0.214858,0.810937
2,US0382221051,2018,6.900000,6.190029,6.683383,8.140347,0.656745,0.230856,1.174702,0.244051
3,US11135F1012,2018,2.030000,0.000000,1.427484,7.473958,0.660284,1.069467,0.703850,-0.617206
4,US29355A1079,2018,3.070000,0.130538,6.115842,5.971274,-0.749523,0.814302,-4.199872,0.463123
...,...,...,...,...,...,...,...,...,...,...
2083,US0395872098,2023,2.359504,0.000000,0.894499,6.970515,1.099504,0.000000,0.088000,1.727402
2084,US54405Q2093,2023,1.979853,0.000000,0.675000,5.924509,0.729853,0.000000,0.092296,0.294171
2085,US3073597037,2023,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
2086,CA35954F1080,2023,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN


In [7]:
historical_score[historical_score['Isin']=='US00108J1097']

,Isin,Year,SR001,SR002,SR003,SR004,Trend_ESG_SR001,Trend_ENV_SR002,Trend_SOC_SR003,Trend_GOV_SR004
21,US00108J1097,2018,NaN,NaN,NaN,NaN,0.600903,0.0,0.222608,0.449479
352,US00108J1097,2019,NaN,NaN,NaN,NaN,0.600903,0.0,0.222608,0.449479
683,US00108J1097,2020,NaN,NaN,NaN,NaN,0.600903,0.0,0.222608,0.449479
1014,US00108J1097,2021,1.430000,0.0,0.857392,5.240198,0.600903,0.0,0.222608,0.449479
1345,US00108J1097,2022,1.470000,0.0,0.857392,5.476806,0.600903,0.0,0.222608,0.449479
1741,US00108J1097,2023,2.030903,0.0,1.080000,5.689677,0.600903,0.0,0.222608,0.449479


In [8]:
historical_score.to_csv('../data/historical_score.csv', index=False)

In [9]:
hist_scope = hist_scope.replace(['#N/A', '#N/A Field Not Applicable'], np.nan)
numeric_columns = ['EG017', 'EG018', 'EG030', 'IS010']
hist_scope[numeric_columns] = hist_scope[numeric_columns].astype(float)
hist_scope

,Isin,Year,EG017,EG018,EG030,IS010
0,US0079031078,2010,26.628,140.384,286.840,6494.000000
1,US0326541051,2010,34.933,96.687,14.211,2761.503000
2,US0382221051,2010,42.172,133.960,36.863,9549.000000
3,US11135F1012,2010,110.810,156.699,10.771,2093.000000
4,US29355A1079,2010,0.005,9.199,14.247,61.661000
...,...,...,...,...,...,...
4867,US0395872098,2023,NaN,NaN,NaN,NaN
4868,US54405Q2093,2023,0.096,0.296,3.404,2.340000
4869,US3073597037,2023,0.016,0.045,0.055,0.784000
4870,CA35954F1080,2023,0.000,0.000,14.043,0.000000


In [11]:
historical_scope= hist_scope
historical_scope.to_csv('../data/historical_scope.csv', index=False)

In [12]:
ESG

,ISIN,Industry,ESG SCORE,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,ESG Score Range,Mean ESG,ESG Classification,E Classification,S Classification,G Classification
0,CNE100004ZD8,Semiconductors,3.045588,3.068010,1.080000,4.981279,3-4,3.764817,Below Median,Leading,Below Median,Leading
1,US00108J1097,Semiconductors,2.030903,0.000000,1.080000,5.689677,2-3,3.764817,Below Median,Below Median,Below Median,Above Median
2,US0079031078,Semiconductors,6.960441,7.325640,5.879457,7.554493,6-7,3.764817,Above Median,Above Median,Above Median,Above Median
3,TW0003438008,Semiconductors,3.179927,1.719228,2.368524,5.938929,3-4,3.764817,Below Median,Below Median,Below Median,Above Median
4,CNE100003MM9,Semiconductors,4.238693,4.181856,4.137878,4.415290,4-5,3.764817,Leading,Above Median,Above Median,Below Median
...,...,...,...,...,...,...,...,...,...,...,...,...
343,US0395872098,Automobiles,2.359504,0.000000,0.894499,6.970515,2-3,2.934915,Below Median,Below Median,Below Median,Above Median
344,US54405Q2093,Automobiles,1.979853,0.000000,0.675000,5.924509,1-2,2.934915,Below Median,Below Median,Below Median,Above Median
345,US3073597037,Automobiles,0.000000,0.000000,0.000000,0.000000,0-1,2.934915,Below Median,Below Median,Below Median,Below Median
346,CA35954F1080,Automobiles,0.000000,0.000000,0.000000,0.000000,0-1,2.934915,Below Median,Below Median,Below Median,Below Median
